In [1]:
import pickle

tweets = []
tmin = 500
tmax = 479500 # latest file name

for i in range(tmin, tmax + 1, 500):
    file_name = "../dataKA/all_tweets/tweets" + str(i)
    file_object = open(file_name, 'rb')
    for tweet in pickle.load(file_object):
        tweets.append(tweet)

In [2]:
keywords = ["covid", "covid19", "covid2019", "covid-19", "covid-2019", "covid_19", "covid_2019", "COVIDー19",
            "COVIDー2019", "corona", "virus", "coronavirus", "coronakrise", "coronacrisis", "corona-crise", 
            "pandemic", "coronapandemic", "coronavirusapandemic", "covidpandemic", "covid19pandemic", "covid2019pandemic" 
            "iorestoacasa", "forzalombardia", "quarantena", "covidswitzerland", "wtfockdown", "stayhome", "stayathome",
            "staysafe", "stayhomesavelives", "stayhomestaysafe", "lockdown", "socialdistancing", "distancing", "quarantine",
            "quarantinelife", "confinement", "confinementjour", "restezchezvous", "lavuedepuismonconfinement", 
            "BloqueoNoSolidaridadSi", "protectyourselfandothers", "coronavirusdelcastigominore", "coronainfoch", "masken",
            "masques", "masks", "homeoffice", "wfh", "workfromhome", "BAG_OFSP_UFSP", "restriktionen"] 


# from Karolos
def check(words, sentence):
    for word in words:
        if word in sentence:
            return True
    return False

keywords_lower = [k.lower() for k in keywords]

In [3]:
# COVID-related
fr_cov_date = []
fr_cov_tid = []
fr_cov_text = []
fr_cov_tweets = []

cnt = 0

for tweet in tweets:
    
    if check(keywords_lower, tweet["text"].lower()):
        if tweet["lang"] == "fr":
            cnt = cnt + 1
            fr_cov_tweets.append(tweet)
            fr_cov_tid.append(tweet["id"])
            fr_cov_date.append(tweet["created_at"])
            try:
                fr_cov_text.append(tweet["extended_tweet"]["full_text"])
            except KeyError:
                fr_cov_text.append(tweet["text"])
            
    elif "extended_tweet" in tweet and check(keywords_lower, tweet["extended_tweet"]["full_text"]):
        if tweet["lang"] == "fr":
            cnt = cnt + 1
            fr_cov_tweets.append(tweet)
            fr_cov_tid.append(tweet["id"])
            fr_cov_date.append(tweet["created_at"])
            fr_cov_text.append(tweet["extended_tweet"]["full_text"])
            
    elif "retweeted_status" in tweet and check(keywords_lower, tweet["retweeted_status"]["full_text"]):
        if tweet["lang"] == "fr":
            cnt = cnt + 1
            fr_cov_tweets.append(tweet)
            fr_cov_tid.append(tweet["id"])
            fr_cov_date.append(tweet["created_at"])
            fr_cov_text.append(tweet["retweeted_status"]["full_text"])
            
    elif "quoted_status" in tweet and check(keywords_lower, tweet["quoted_status"]["text"]):
        if tweet["lang"] == "fr":
            cnt = cnt + 1
            fr_cov_tweets.append(tweet)
            fr_cov_tid.append(tweet["id"])
            fr_cov_date.append(tweet["created_at"])
            fr_cov_text.append(tweet["quoted_status"]["text"])

print("No. FR covid tweets: ", cnt)

No. FR covid tweets:  7485


In [4]:
# All FR tweets
fr_all_date = []
fr_all_tid = []
fr_all_text = []
fr_all_tweets = []

for tweet in tweets:
    if tweet['lang'] == 'fr':
        fr_all_tweets.append(tweet)
        fr_all_tid.append(tweet["id"])
        fr_all_date.append(tweet["created_at"])
        try:
            fr_all_text.append(tweet["extended_tweet"]["full_text"])
        except KeyError:
            fr_all_text.append(tweet["text"])

print("No. FR tweets (total): ", len(fr_all_tid))

No. FR tweets (total):  107978


In [5]:
import pandas as pd
import numpy as np


## Export full tweet datasets ##

fr_all = pd.concat([pd.Series(fr_all_tid), 
                           pd.Series(fr_all_date), 
                           pd.Series(fr_all_text)],
                          axis = 1)
fr_covid = pd.concat([pd.Series(fr_cov_tid),
                             pd.Series(fr_cov_date),
                             pd.Series(fr_cov_text)],
                            axis = 1)

# fr_all.to_csv("../dataKA/latest/fr_tweets_all.csv", encoding = "utf-8")
# fr_covid.to_csv("../dataKA/latest/fr_tweets_covid.csv", encoding = "utf-8")

## Export text of covid dataset and full dataset for tool training ## 
out_fr_all_text = pd.DataFrame(fr_all_text)
out_fr_cov_text = pd.DataFrame(fr_cov_text)

# out_fr_all_text.to_csv("../dataKA/latest/fr_alltext.csv", encoding = "utf-8")
# out_fr_cov_text.to_csv("../dataKA/latest/fr_covidtext.csv", encoding = "utf-8")

In [6]:
import re
import string
from collections import Counter

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV


#### Define pre-processing subfunctions ####

def removeNonAlphaNumChars(tweets):
    tweets = [re.sub(r'\-', r' ', tweet) for tweet in tweets]  # replace - with SPACE
    tweets = [re.sub(r'[^\w\s]', r'', tweet) for tweet in tweets]  #remove other NON alpha numeric, excluding whitespace
    return tweets

def replaceContractions(tweets):
    tweets = [re.sub(r"[’\']", r' ', tweet) for tweet in tweets]  ## !!!! WEIRD APOSTROPHE BEWARE !!!
    return tweets

# FROM DEFFRO: https://github.com/Deffro/text-preprocessing-techniques
def removeUnicode(tweets):
    """ Removes unicode strings like "\u002c" and "x96" """
    tweets = [re.sub(r'(\\u[0-9A-Fa-f]+)',r'', tweet) for tweet in tweets]
    tweets = [re.sub(r'[^\x00-\x7f]',r'',tweet) for tweet in tweets]
    return tweets

def replaceURL(tweets):
    """ Replaces url address with "url" """
    tweets = [re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', tweet) for tweet in tweets]
    tweets = [re.sub(r'#([^\s]+)', r'\1', tweet) for tweet in tweets]
    return tweets

def replaceAtUser(tweets):
    """ Replaces "@user" with "atUser" """
    tweets = [re.sub('@[^\s]+','atUser',tweet) for tweet in tweets]
    return tweets

def removeHashtagInFrontOfWord(tweets):
    """ Removes hastag in front of a word """
    tweets = [re.sub(r'#([^\s]+)', r'\1', tweet) for tweet in tweets]
    return tweets
# end DEFFRO


#### DEFINE MAIN PREPROCESSING FUNCTIONS ####

def cleanTweets(tweets):
    tweets = [tweet.lower() for tweet in tweets]  #convert to lowercase
    tweets = replaceURL(tweets) #replace URLs
    tweets = replaceAtUser(tweets)  # replace @user
    tweets = removeHashtagInFrontOfWord(tweets)  # remove hashtag
    tweets = replaceContractions(tweets) # replace ' contractions with space (e.g. l'ordre = l ordre)
    tweets = removeNonAlphaNumChars(tweets)  # remove non alphanumeric characters
    return tweets

def tokenizeTweets(tweets):
    tokens = [nltk.word_tokenize(tweet) for tweet in tweets]
    return tokens

def applyStoplist(tokens):
    tokens = [[word for word in token if len(word) > 2 and (word not in stoplist)] for token in tokens]
    return tokens

def stemTokens(tokens):
    tokens = [[stemmer.stem(word) for word in token] for token in tokens]
    return tokens


In [9]:
# Import preclassified EN tweets (football)
fr_preClass = pd.read_csv("C:/Users/pang/Desktop/Stuff/_PhD/PhD Courses/_CSM/Project/preClassified/FR_3cat_worldcup.csv")
# fr_preClass = shuffle(fr_preClass)

neutral = fr_preClass[fr_preClass["sentiment"]=="NEUTRAL"]
positive = fr_preClass[fr_preClass["sentiment"]=="POSITIVE"]
negative = fr_preClass[fr_preClass["sentiment"]=="NEGATIVE"]

neutral = shuffle(neutral)[:20000]

print("No. neutral: ", len(neutral))
print("No. positive: ", len(positive))
print("No. negative: ", len(negative))

No. neutral:  20000
No. positive:  14328
No. negative:  2358


In [10]:
## Start tweet preprocessing ##
stoplist = stopwords.words('french')
other_stopwords = "atUser URL"
stoplist = stoplist + other_stopwords.split() # final list of stopwords
# lemmatizer = WordNetLemmatizer() # only for EN
stemmer = SnowballStemmer('french') # set stemmer

fr_preClass_pos_clean = cleanTweets(positive["tweet_text"])
fr_preClass_pos_tokens = tokenizeTweets(fr_preClass_pos_clean)
fr_preClass_pos_tokens = applyStoplist(fr_preClass_pos_tokens)
fr_preClass_pos_tokens = stemTokens(fr_preClass_pos_tokens)

fr_preClass_neg_clean = cleanTweets(negative["tweet_text"])
fr_preClass_neg_tokens = tokenizeTweets(fr_preClass_neg_clean)
fr_preClass_neg_tokens = applyStoplist(fr_preClass_neg_tokens)
fr_preClass_neg_tokens = stemTokens(fr_preClass_neg_tokens)

fr_preClass_neu_clean = cleanTweets(neutral["tweet_text"])
fr_preClass_neu_tokens = tokenizeTweets(fr_preClass_neu_clean)
fr_preClass_neu_tokens = applyStoplist(fr_preClass_neu_tokens)
fr_preClass_neu_tokens = stemTokens(fr_preClass_neu_tokens)

In [11]:
fr_pos_labeled = [[token, "Positive"] for token in fr_preClass_pos_tokens]
fr_neg_labeled = [[token, "Negative"] for token in fr_preClass_neg_tokens]
fr_neu_labeled = [[token, "Neutral"] for token in fr_preClass_neu_tokens]

fr_dataset = fr_pos_labeled + fr_neg_labeled + fr_neu_labeled
fr_dataset = shuffle(pd.DataFrame(fr_dataset))

# split features and labels for SKLEARN classifier
features = fr_dataset.loc[:, 0]
labels  = fr_dataset.loc[:, 1]

## Use in-domain test set:

In [12]:
## Prepare for classifier training (SKLEARN) ##

def dummy(doc):
    return doc

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=dummy,preprocessor=dummy,max_df=0.9, min_df=10)
X=cv.fit_transform(features).toarray()
y=labels

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)

In [13]:
## NAIVE BAYES USING SKLEARN ##

#Import Multinomial Naive Bayes model (https://towardsdatascience.com/sentiment-analysis-of-tweets-using-multinomial-naive-bayes-1009ed24276b)
from sklearn.naive_bayes import MultinomialNB

#Create a multinomial NB Classifier
mnb = MultinomialNB()

#Train the model using the training sets
mnb.fit(X_train, y_train)

#Predict the response for test dataset
y_pred_NB = mnb.predict(X_test)

In [17]:
# ## RANDOM FOREST USING SKLEARN ##

# # Fitting classifier to the Training set
# from sklearn.ensemble import RandomForestClassifier

# RFclassifier=RandomForestClassifier(n_estimators=200, random_state=0)
# RFclassifier.fit(X_train,y_train)
# # Predicting the Test set results
# y_pred_RF = RFclassifier.predict(X_test)

In [14]:
from sklearn import metrics
from sklearn.metrics import classification_report

print (classification_report(y_test, y_pred_NB))
print("\nAccuracy (NB):", metrics.accuracy_score(y_test, y_pred_NB))

# print ("\n", classification_report(y_test, y_pred_RF))
# print("\nAccuracy (RF):", metrics.accuracy_score(y_test, y_pred_RF))

              precision    recall  f1-score   support

    Negative       0.34      0.58      0.43       736
     Neutral       0.76      0.63      0.68      5982
    Positive       0.64      0.72      0.68      4288

    accuracy                           0.66     11006
   macro avg       0.58      0.64      0.60     11006
weighted avg       0.68      0.66      0.67     11006


Accuracy (NB): 0.6597310557877522


## Use collected tweets as test set instead of in-domain tweets:

In [15]:
### load manually classified tweets (CH/Covid) ###

fr_labeled_all = pd.read_csv("../dataKA/frlabeledAll1.csv")
fr_labeled_cov = pd.read_csv("../dataKA/frlabeledCovid1.csv")

## ALL ##
neu = fr_labeled_all[fr_labeled_all["label"]=="neu"]
pos = fr_labeled_all[fr_labeled_all["label"]=="pos"]
neg = fr_labeled_all[fr_labeled_all["label"]=="neg"]
print("No. neutral (all): ", len(neu))
print("No. positive (all): ", len(pos))
print("No. negative (all): ", len(neg))

# Start preprocessing #
fr_labeled_all_pos_clean = cleanTweets(pos["text"])
fr_labeled_all_pos_tokens = tokenizeTweets(fr_labeled_all_pos_clean)
fr_labeled_all_pos_tokens = applyStoplist(fr_labeled_all_pos_tokens)
fr_labeled_all_pos_tokens = stemTokens(fr_labeled_all_pos_tokens)

fr_labeled_all_neg_clean = cleanTweets(neg["text"])
fr_labeled_all_neg_tokens = tokenizeTweets(fr_labeled_all_neg_clean)
fr_labeled_all_neg_tokens = applyStoplist(fr_labeled_all_neg_tokens)
fr_labeled_all_neg_tokens = stemTokens(fr_labeled_all_neg_tokens)

fr_labeled_all_neu_clean = cleanTweets(neu["text"])
fr_labeled_all_neu_tokens = tokenizeTweets(fr_labeled_all_neu_clean)
fr_labeled_all_neu_tokens = applyStoplist(fr_labeled_all_neu_tokens)
fr_labeled_all_neu_tokens = stemTokens(fr_labeled_all_neu_tokens)

fr_pos_labeled_all = [[token, "Positive"] for token in fr_labeled_all_pos_tokens]
fr_neg_labeled_all = [[token, "Negative"] for token in fr_labeled_all_neg_tokens]
fr_neu_labeled_all = [[token, "Neutral"] for token in fr_labeled_all_neu_tokens]

fr_dataset_all = fr_pos_labeled_all + fr_neg_labeled_all + fr_neu_labeled_all
fr_dataset_all = shuffle(pd.DataFrame(fr_dataset_all))


## COVID ##
neu_cov = fr_labeled_cov[fr_labeled_cov["label"]=="neu"]
pos_cov = fr_labeled_cov[fr_labeled_cov["label"]=="pos"]
neg_cov = fr_labeled_cov[fr_labeled_cov["label"]=="neg"]
print("No. neutral (covid): ", len(neu_cov))
print("No. positive (covid): ", len(pos_cov))
print("No. negative (covid): ", len(neg_cov))

## Start preprocessing ##
fr_labeled_cov_pos_clean = cleanTweets(pos_cov["text"])
fr_labeled_cov_pos_tokens = tokenizeTweets(fr_labeled_cov_pos_clean)
fr_labeled_cov_pos_tokens = applyStoplist(fr_labeled_cov_pos_tokens)
fr_labeled_cov_pos_tokens = stemTokens(fr_labeled_cov_pos_tokens)

fr_labeled_cov_neg_clean = cleanTweets(neg_cov["text"])
fr_labeled_cov_neg_tokens = tokenizeTweets(fr_labeled_cov_neg_clean)
fr_labeled_cov_neg_tokens = applyStoplist(fr_labeled_cov_neg_tokens)
fr_labeled_cov_neg_tokens = stemTokens(fr_labeled_cov_neg_tokens)

fr_labeled_cov_neu_clean = cleanTweets(neu_cov["text"])
fr_labeled_cov_neu_tokens = tokenizeTweets(fr_labeled_cov_neu_clean)
fr_labeled_cov_neu_tokens = applyStoplist(fr_labeled_cov_neu_tokens)
fr_labeled_cov_neu_tokens = stemTokens(fr_labeled_cov_neu_tokens)

fr_pos_labeled_cov = [[token, "Positive"] for token in fr_labeled_cov_pos_tokens]
fr_neg_labeled_cov = [[token, "Negative"] for token in fr_labeled_cov_neg_tokens]
fr_neu_labeled_cov = [[token, "Neutral"] for token in fr_labeled_cov_neu_tokens]

fr_dataset_cov = fr_pos_labeled_cov + fr_neg_labeled_cov + fr_neu_labeled_cov
fr_dataset_cov = shuffle(pd.DataFrame(fr_dataset_cov))


## split features and labels for SKLEARN classifier
features_all = fr_dataset_all.loc[:, 0]
labels_all  = fr_dataset_all.loc[:, 1]
features_cov = fr_dataset_cov.loc[:, 0]
labels_cov  = fr_dataset_cov.loc[:, 1]

features1 = pd.concat([features_all, features], axis=0)
labels1  = pd.concat([labels_all, labels], axis=0)

features2 = pd.concat([features_cov, features], axis=0)
labels2  = pd.concat([labels_cov, labels], axis=0)

print("No. features (preclassified): ", len(features))
print("No. features (all): ", len(features1)-len(features))
print("No. features (covid): ", len(features2)-len(features))

No. neutral (all):  371
No. positive (all):  75
No. negative (all):  54
No. neutral (covid):  414
No. positive (covid):  44
No. negative (covid):  42
No. features (preclassified):  36686
No. features (all):  500
No. features (covid):  500


In [16]:
## use manually labelled tweets (ALL) only as test set ##

X1=cv.fit_transform(features1).toarray()
y1=labels1
# X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size = 249, shuffle = "FALSE", stratify=None)

test_all_index = 500 # USE THIS INSTEAD OF SKLEARN HELPER TO ENSURE CORRECT SPLIT (first 500 are collected tweets)
X_test1, X_train1 = X1[:test_all_index], X1[test_all_index:]
y_test1, y_train1 = y1[:test_all_index], y1[test_all_index:]
print("Size (test set): ", len(X_test1))

# sklearn multinomial NB classifier #
mnb.fit(X_train1, y_train1)
y_pred_NB1 = mnb.predict(X_test1)
print ("\n", classification_report(y_test1, y_pred_NB1))
print("\nAccuracy (NB) of manually classified tweets, all:", metrics.accuracy_score(y_test1, y_pred_NB1))

# # sklearn random forest classfier #
# RFclassifier.fit(X_train1,y_train1)
# y_pred_RF1 = RFclassifier.predict(X_test1)
# print ("\n", classification_report(y_test1, y_pred_RF1))
# print("\nAccuracy (RF) of manually classified tweets, all:", metrics.accuracy_score(y_test1, y_pred_RF1))

Size (test set):  500

               precision    recall  f1-score   support

    Negative       0.27      0.50      0.35        54
     Neutral       0.83      0.63      0.71       371
    Positive       0.27      0.44      0.34        75

    accuracy                           0.58       500
   macro avg       0.46      0.52      0.47       500
weighted avg       0.69      0.58      0.62       500


Accuracy (NB) of manually classified tweets, all: 0.584


In [20]:
## use manually labelled tweets (COVID) only as test set ##

X2=cv.fit_transform(features2).toarray()
y2=labels2
# X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size = 250, shuffle = "FALSE", stratify=None)

test_cov_index = 500 # USE THIS INSTEAD OF SKLEARN HELPER TO ENSURE CORRECT SPLIT
X_test2, X_train2 = X2[:test_cov_index], X2[test_cov_index:]
y_test2, y_train2 = y2[:test_cov_index], y2[test_cov_index:]
print("Size (test set): ", len(X_test2))
print(y_test2[:3])

# sklearn multinomial NB classifier #
mnb.fit(X_train2, y_train2)
y_pred_NB2 = mnb.predict(X_test2)
print ("\n", classification_report(y_test2, y_pred_NB2))
print("\nAccuracy (NB) of manually classified tweets, covid-related:", metrics.accuracy_score(y_test2, y_pred_NB2))

# # sklearn random forest classfier #
# RFclassifier.fit(X_train2, y_train2)
# y_pred_RF2 = RFclassifier.predict(X_test2)
# print ("\n", classification_report(y_test2, y_pred_RF2))
# print("\nAccuracy (RF) of manually classified tweets, all:", metrics.accuracy_score(y_test2, y_pred_RF2))

Size (test set):  500
497     Neutral
143     Neutral
52     Negative
Name: 1, dtype: object

               precision    recall  f1-score   support

    Negative       0.13      0.74      0.22        42
     Neutral       0.90      0.41      0.56       414
    Positive       0.36      0.61      0.46        44

    accuracy                           0.46       500
   macro avg       0.47      0.59      0.41       500
weighted avg       0.79      0.46      0.53       500


Accuracy (NB) of manually classified tweets, covid-related: 0.456


## Apply trained tool to predict polarity of collected FR tweets

In [18]:
# Using sklearn mNB classifier
labels_all = pd.Series(["tbc"] * len(fr_all_tweets))
features_all = cleanTweets(fr_all_text)
features_all = tokenizeTweets(features_all)
features_all = applyStoplist(features_all)
features_all = stemTokens(features_all)
features_all = pd.Series(features_all)
labels_train = labels
features_train = features

X_all = cv.fit_transform(pd.concat([features_train, features_all], axis=0)).toarray()
y_all = pd.concat([labels_train, labels_train], axis=0)

cv.fit_transform(features).toarray()
train_index = len(features_train) # USE THIS INSTEAD OF SKLEARN HELPER TO ENSURE CORRECT SPLIT
X_all_train, X_all_test = X_all[:train_index], X_all[train_index:]
y_all_train, y_all_test = y_all[:train_index], y_all[train_index:]

print("Size (test set): ", len(X_all_test))

Size (test set):  107978


In [19]:
# Naive Bayes
mnb.fit(X_all_train, y_all_train)
y_all_pred_NB = mnb.predict(X_all_test)
fr_res_all_NB = pd.concat([pd.Series(fr_all_tid), pd.Series(fr_all_date),
                     pd.Series(fr_all_text), pd.Series(y_all_pred_NB)], 
                    axis = 1) 
fr_res_all_NB.to_csv("../dataKA/latest/fr_res_all_nb.csv", encoding = "utf-8")
print("No. predictions: ", len(y_all_pred_NB))
print(y_all_pred_NB[:10])
print(fr_res_all_NB[:5])

No. predictions:  107978
['Positive' 'Positive' 'Neutral' 'Positive' 'Neutral' 'Neutral' 'Negative'
 'Neutral' 'Neutral' 'Negative']
                     0                               1  \
0  1244238235351556098  Sun Mar 29 12:21:26 +0000 2020   
1  1244238255916228608  Sun Mar 29 12:21:31 +0000 2020   
2  1244238328943259649  Sun Mar 29 12:21:49 +0000 2020   
3  1244238523642847234  Sun Mar 29 12:22:35 +0000 2020   
4  1244238528025964544  Sun Mar 29 12:22:36 +0000 2020   

                                                   2         3  
0                           @clemd95 Serieux ?! 😱😱😱😱  Positive  
1  Mirabelle incoming! 😋 @ Baume-les-Dames https:...  Positive  
2                                @AkaDiane09 Fume le   Neutral  
3  🇪🇺 L'Europe des solidarités.🇪🇺\n\nMerci aux Al...  Positive  
4  @charlotteepenex @AlexGubser @BaselStadt @Etat...   Neutral  


In [ ]:
# # Random forest
# from sklearn.ensemble import RandomForestClassifier
# RFclassifier=RandomForestClassifier(n_estimators=200, random_state=0)

# RFclassifier.fit(X_all_train, y_all_train)
# y_all_pred_RF2 = RFclassifier.predict(X_all_test)
# res_all_RF = pd.concat([pd.Series(fr_all_tweet_id), pd.Series(fr_all_date), 
#                      pd.Series(fr_all_coord), pd.Series(fr_all_place), 
#                      pd.Series(fr_all_text), pd.Series(y_all_pred_RF2)], 
#                     axis = 1) 
# res_all_RF.to_csv("../dataKA/res_all_FR_rf.csv", encoding = "utf-8")
# print("No. predictions: ", len(y_all_pred_RF2))
# print(res_all_RF[:5])